# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [118]:
# import libraries
import os
import re
import numpy as np
import pandas as pd
import pickle
from time import time

# import nltk libraries
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# import sklearn libraries
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV 
from sklearn.svm import LinearSVC
from sklearn.multioutput import MultiOutputClassifier


from sqlalchemy import create_engine

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Peter\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Peter\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Peter\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [119]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('disaster_table', con=engine)

X = df['message'] # reads message column
y = df.iloc[:,4:] # reads all rows, all column starting at [4] ->

In [120]:
# Display message column
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [121]:
# Display categories dataframe made from ETL pipeline
y.head()

,related,request,offer,aid related,medical help,medical products,search and rescue,security,military,child alone,...,aid centers,other infrastructure,weather related,floods,storm,fire,earthquake,cold,other weather,direct report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [122]:
# Displays length of characters in each message
df['message'] = df['message'].apply(lambda x: len(x))
df['message']

0         66
1         39
2         31
3        100
4         63
        ... 
26210    148
26211    136
26212    141
26213    191
26214    209
Name: message, Length: 26215, dtype: int64

In [123]:
# Index of column names
y.columns

Index(['related', 'request', 'offer', 'aid related', 'medical help',
       'medical products', 'search and rescue', 'security', 'military',
       'child alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing people', 'refugees', 'death', 'other aid',
       'infrastructure related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid centers', 'other infrastructure',
       'weather related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other weather', 'direct report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [124]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

# function to detect url, normalize, tokenize, remove stopwords, lemmatize words 
def tokenize(text, lemmatizer=WordNetLemmatizer()):
    # Detect urls with url_regex
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    # Normalizes and tokenizes (removes punctuations and converts to lowercase)
    tokens = nltk.word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower()))

        # Remove stopwords
    tokens = [t for t in tokens if t not in stopwords.words('english')]

    # lemmatize and remove stop words
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        if clean_tok not in stopwords.words("english"):
            clean_tokens.append(clean_tok)
    pass
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [125]:
# Pipeline with RandomForestClf
pipeline = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier(n_jobs=-1))),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [126]:
# Splits dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [127]:
# Fits pipeline on train set 
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x0000020082A9FDC0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))])

In [128]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   TfidfVectorizer(tokenizer=<function tokenize at 0x0000020082A9FDC0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))],
 'verbose': False,
 'vect': TfidfVectorizer(tokenizer=<function tokenize at 0x0000020082A9FDC0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.float64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__norm': 'l2',
 'vect__preprocessor': None,
 'vect__smooth_idf': True,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__sublinear_tf': False,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text, lemmatizer=<

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [129]:
# Predict pipeline X_test (long process)
y_pred = pipeline.predict(X_test)

In [130]:
print(classification_report(y_test, y_pred, target_names=y.columns))

                        precision    recall  f1-score   support

               related       0.84      0.94      0.89      3998
               request       0.83      0.51      0.63       891
                 offer       0.00      0.00      0.00        24
           aid related       0.76      0.69      0.72      2164
          medical help       0.67      0.07      0.12       435
      medical products       0.79      0.08      0.14       279
     search and rescue       0.62      0.07      0.13       136
              security       0.33      0.01      0.02        96
              military       0.57      0.08      0.14       158
           child alone       0.00      0.00      0.00         0
                 water       0.89      0.43      0.58       335
                  food       0.87      0.54      0.66       584
               shelter       0.80      0.36      0.50       468
              clothing       0.89      0.11      0.20        70
                 money       0.90      

### 6. Improve your model
Use grid search to find better parameters. 

In [131]:
# Pipeline 2 with RFC
pipeline2 = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier())),
])

In [132]:
from sklearn.model_selection import GridSearchCV

# Parameters used for GridsearchCV
parameters2 = {'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [10],}

cv2 = GridSearchCV(estimator=pipeline2, param_grid=parameters2, verbose=3)

In [133]:
cv2.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__verbose', 'estimator__vect', 'estimator__tfidf', 'estimator__clf', 'estimator__vect__analyzer', 'estimator__vect__binary', 'estimator__vect__decode_error', 'estimator__vect__dtype', 'estimator__vect__encoding', 'estimator__vect__input', 'estimator__vect__lowercase', 'estimator__vect__max_df', 'estimator__vect__max_features', 'estimator__vect__min_df', 'estimator__vect__ngram_range', 'estimator__vect__norm', 'estimator__vect__preprocessor', 'estimator__vect__smooth_idf', 'estimator__vect__stop_words', 'estimator__vect__strip_accents', 'estimator__vect__sublinear_tf', 'estimator__vect__token_pattern', 'estimator__vect__tokenizer', 'estimator__vect__use_idf', 'estimator__vect__vocabulary', 'estimator__tfidf__norm', 'estimator__tfidf__smooth_idf', 'estimator__tfidf__sublinear_tf', 'estimator__tfidf__use_idf', 'estimator__clf__estimator__bootstrap', 'estimator__clf__estimator__ccp_alpha', 'estimator__clf__e

In [134]:
cv2

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(tokenizer=<function tokenize at 0x0000020082A9FDC0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [10],
                         'tfidf__use_idf': (True, False)},
             verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [135]:
cv2.fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True ............
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, score=0.247, total= 8.2min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True ............
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  8.2min remaining:    0.0s
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, score=0.251, total= 8.6min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True ............
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 16.8min remaining:    0.0s
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, score=0.245, total= 7.5min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True ............
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, score=0.247, total= 7.5min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True .........

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(tokenizer=<function tokenize at 0x0000020082A9FDC0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [10],
                         'tfidf__use_idf': (True, False)},
             verbose=3)

In [136]:
y_pred2 = cv2.predict(X_test)

In [137]:
print(classification_report(y_test, y_pred2, target_names=y.columns))

                        precision    recall  f1-score   support

               related       0.85      0.91      0.88      3998
               request       0.79      0.42      0.55       891
                 offer       0.00      0.00      0.00        24
           aid related       0.74      0.60      0.66      2164
          medical help       0.57      0.09      0.15       435
      medical products       0.77      0.06      0.11       279
     search and rescue       0.69      0.08      0.14       136
              security       0.00      0.00      0.00        96
              military       0.62      0.08      0.15       158
           child alone       0.00      0.00      0.00         0
                 water       0.81      0.32      0.46       335
                  food       0.84      0.43      0.56       584
               shelter       0.79      0.35      0.49       468
              clothing       0.67      0.09      0.15        70
                 money       0.55      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [153]:
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier

# Pipeline 3 with AdaboostClf
pipeline3 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer(smooth_idf=False)),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [154]:
pipeline3.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000020082A9FDC0>)),
  ('tfidf', TfidfTransformer(smooth_idf=False)),
  ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000020082A9FDC0>),
 'tfidf': TfidfTransformer(smooth_idf=False),
 'clf': MultiOutputClassifier(estimator=AdaBoostClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text, lemmatizer=<WordNetLemmatizer>)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 't

In [155]:
# tuning parameters {'clf__estimator__n_estimators': [10, 30]
parameters3 = {'clf__estimator__n_estimators': [50, 100], 
              'clf__estimator__learning_rate': [1, 2]}


cv3 = GridSearchCV(estimator=pipeline3, param_grid=parameters3, verbose=3)

In [156]:
cv3

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000020082A9FDC0>)),
                                       ('tfidf',
                                        TfidfTransformer(smooth_idf=False)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'clf__estimator__learning_rate': [1, 2],
                         'clf__estimator__n_estimators': [50, 100]},
             verbose=3)

In [157]:
cv3.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=50 
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=50, score=0.242, total= 8.4min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=50 
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  8.4min remaining:    0.0s
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=50, score=0.240, total= 8.1min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=50 
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 16.5min remaining:    0.0s
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=50, score=0.248, total= 8.1min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=50 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=50, score=0.242, total= 7.9min
[CV] clf__estimator

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000020082A9FDC0>)),
                                       ('tfidf',
                                        TfidfTransformer(smooth_idf=False)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'clf__estimator__learning_rate': [1, 2],
                         'clf__estimator__n_estimators': [50, 100]},
             verbose=3)

In [158]:
y_pred3 = cv3.predict(X_test)

In [159]:
# cv3 best parameters
print(classification_report(y_test, y_pred3, target_names=y.columns))

                        precision    recall  f1-score   support

               related       0.82      0.95      0.88      3998
               request       0.78      0.53      0.63       891
                 offer       0.00      0.00      0.00        24
           aid related       0.78      0.62      0.69      2164
          medical help       0.60      0.26      0.36       435
      medical products       0.59      0.30      0.40       279
     search and rescue       0.47      0.15      0.22       136
              security       0.17      0.04      0.07        96
              military       0.53      0.32      0.40       158
           child alone       0.00      0.00      0.00         0
                 water       0.73      0.68      0.70       335
                  food       0.81      0.68      0.74       584
               shelter       0.74      0.58      0.65       468
              clothing       0.74      0.40      0.52        70
                 money       0.49      

### 9. Export your model as a pickle file

In [165]:
p = pickle.dumps('classifier.pkl')

In [166]:
with open('classifier.pkl', 'wb') as file:
    pickle.dump(cv3, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.